In [1]:
import os
import time
import numpy as np
import pandas as pd
import scipy.io as sio
from IPython.display import display
import itertools as it

import matplotlib.pyplot as plt
import pywt
import scipy.stats

import datetime as dt
from collections import defaultdict, Counter

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.gaussian_process import GaussianProcessClassifier

In [2]:
data1=pd.read_csv('Accel Data 01.csv')
data2=pd.read_csv('Accel Data 02.csv')
data3=pd.read_csv('Accel Data 03.csv')
data4=pd.read_csv('Accel Data 04.csv')
data5=pd.read_csv('Accel Data 05.csv')
data6=pd.read_csv('Accel Data 06.csv')


In [3]:
data_cols = ['time', 'accelx', 'accely','accelz', 'absolute_accel']
data1.columns = data_cols
data2.columns = data_cols
data3.columns = data_cols
data4.columns = data_cols
data5.columns = data_cols
data6.columns = data_cols

In [4]:
step11 = data1[data1.time<=20.0]
step21 = data2[data2.time<=20.0]
step31 = data3[data3.time<=20.0]
step41 = data4[data4.time<=20.0]
step51 = data5[data5.time<=20.0]
step61 = data6[data6.time<=20.0]
#STEP 1 for all data

In [5]:
#STEP 2
step12 = data1[(data1.time>20.0) & (data1.time<=40.0)]
step22 = data2[(data2.time>20.0) & (data2.time<=40.0)]
step32 = data3[(data3.time>20.0) & (data3.time<=40.0)]
step42 = data4[(data4.time>20.0) & (data4.time<=40.0)]
step52 = data5[(data5.time>20.0) & (data5.time<=40.0)]
step62 = data6[(data6.time>20.0) & (data6.time<=40.0)]

In [6]:
#STEP 3
step13 = data1[(data1.time>40.0) & (data1.time<=60.0)]
step23 = data2[(data2.time>40.0) & (data2.time<=60.0)]
step33 = data3[(data3.time>40.0) & (data3.time<=60.0)]
step43 = data4[(data4.time>40.0) & (data4.time<=60.0)]
step53 = data5[(data5.time>40.0) & (data5.time<=60.0)]
step63 = data6[(data6.time>40.0) & (data6.time<=60.0)]

In [7]:
#STEP 4
step14 = data1[(data1.time>60.0) & (data1.time<=80.0)]
step24 = data2[(data2.time>60.0) & (data2.time<=80.0)]
step34 = data3[(data3.time>60.0) & (data3.time<=80.0)]
step44 = data4[(data4.time>60.0) & (data4.time<=80.0)]
step54 = data5[(data5.time>60.0) & (data5.time<=80.0)]
step64 = data6[(data6.time>60.0) & (data6.time<=80.0)]

In [8]:
#STEP 5
step15 = data1[(data1.time>80.0) & (data1.time<=100.0)]
step25 = data2[(data2.time>80.0) & (data2.time<=100.0)]
step35 = data3[(data3.time>80.0) & (data3.time<=100.0)]
step45 = data4[(data4.time>80.0) & (data4.time<=100.0)]
step55 = data5[(data5.time>80.0) & (data5.time<=100.0)]
step65 = data6[(data6.time>80.0) & (data6.time<=100.0)]

In [9]:
#STEP 6
step16 = data1[(data1.time>100.0) & (data1.time<=120.0)]
step26 = data2[(data2.time>100.0) & (data2.time<=120.0)]
step36 = data3[(data3.time>100.0) & (data3.time<=120.0)]
step46 = data4[(data4.time>100.0) & (data4.time<=120.0)]
step56 = data5[(data5.time>100.0) & (data5.time<=120.0)]
step66 = data6[(data6.time>100.0) & (data6.time<=120.0)]

In [10]:
#windowing 

def moving_window(acceld, length, step=1):
    streams = it.tee(acceld, length)
    return zip(*[it.islice(stream, i, None, step*length) for stream, i in zip(streams, it.count(step=step))])


# len(accelx1_) =129 = length in overlap

def overlap(acceld_,length):
    newd=[]
    wd=[]
    for i in range ((length-1)):
        newd=np.concatenate((acceld_[i], acceld_[i+1]))        
        i=i+1
        wd.append(newd)
        
    return wd

def final_window(step):
#     step=stepd.accelx
    step_= list(moving_window(step, 64))
    step_=np.asarray(step_)
    l = len(step_)
    wstep=[]
    wstep=overlap(step_,l)
    wstep=np.asarray(wstep)
    return wstep
    
    
def concat(sub_step):
    wstep=[]
    wstep.concatenate(sub_step)
    return wstep



In [12]:
## STEP 1 x axis
wstep11x=[]
wstep11x=final_window(step11.accelx)
wstep11x.shape

wstep21x=[]
wstep21x=final_window(step21.accelx)

wstep31x=[]
wstep31x=final_window(step31.accelx)

wstep41x=[]
wstep41x=final_window(step41.accelx)

wstep51x=[]
wstep51x=final_window(step51.accelx)

wstep61x=[]
wstep61x=final_window(step61.accelx)

#############################################

wstep11x=np.asarray(wstep11x)
wstep21x=np.asarray(wstep21x)
wstep31x=np.asarray(wstep31x)
wstep41x=np.asarray(wstep41x)
wstep51x=np.asarray(wstep51x)
wstep61x=np.asarray(wstep61x)
wstep1x = []
wstep1x=np.concatenate((wstep11x, wstep21x))
wstep1x=np.concatenate((wstep1x, wstep31x))
wstep1x=np.concatenate((wstep1x, wstep41x))
wstep1x=np.concatenate((wstep1x, wstep51x))
wstep1x=np.concatenate((wstep1x, wstep61x))
wstep1x.shape

## STEP 1 y axis
wstep11y=[]
wstep11y=final_window(step11.accely)
wstep11y.shape

wstep21y=[]
wstep21y=final_window(step21.accely)

wstep31y=[]
wstep31y=final_window(step31.accely)

wstep41y=[]
wstep41y=final_window(step41.accely)

wstep51y=[]
wstep51y=final_window(step51.accely)

wstep61y=[]
wstep61y=final_window(step61.accely)

################################################

wstep11y=np.asarray(wstep11y)
wstep21y=np.asarray(wstep21y)
wstep31y=np.asarray(wstep31y)
wstep41y=np.asarray(wstep41y)
wstep51y=np.asarray(wstep51y)
wstep61y=np.asarray(wstep61y)
wstep1y = []
wstep1y=np.concatenate((wstep11y, wstep21y))
wstep1y=np.concatenate((wstep1y, wstep31y))
wstep1y=np.concatenate((wstep1y, wstep41y))
wstep1y=np.concatenate((wstep1y, wstep51y))
wstep1y=np.concatenate((wstep1y, wstep61y))
wstep1y.shape

## STEP 1 z axis
wstep11z=[]
wstep11z=final_window(step11.accelz)
wstep11z.shape

wstep21z=[]
wstep21z=final_window(step21.accelz)

wstep31z=[]
wstep31z=final_window(step31.accelz)

wstep41z=[]
wstep41z=final_window(step41.accelz)

wstep51z=[]
wstep51z=final_window(step51.accelz)

wstep61z=[]
wstep61z=final_window(step61.accelz)

################################################

wstep11z=np.asarray(wstep11z)
wstep21z=np.asarray(wstep21z)
wstep31z=np.asarray(wstep31z)
wstep41z=np.asarray(wstep41z)
wstep51z=np.asarray(wstep51z)
wstep61z=np.asarray(wstep61z)
wstep1z = []
wstep1z=np.concatenate((wstep11z, wstep21z))
wstep1z=np.concatenate((wstep1z, wstep31z))
wstep1z=np.concatenate((wstep1z, wstep41z))
wstep1z=np.concatenate((wstep1z, wstep51z))
wstep1z=np.concatenate((wstep1z, wstep61z))
wstep1z.shape

In [16]:
## STEP 2 x axis
wstep12x=[]
wstep12x=final_window(step12.accelx)
wstep12x.shape

wstep22x=[]
wstep22x=final_window(step22.accelx)

wstep32x=[]
wstep32x=final_window(step32.accelx)

wstep42x=[]
wstep42x=final_window(step42.accelx)

wstep52x=[]
wstep52x=final_window(step52.accelx)

wstep62x=[]
wstep62x=final_window(step62.accelx)

#############################################

wstep12x=np.asarray(wstep12x)
wstep22x=np.asarray(wstep22x)
wstep32x=np.asarray(wstep32x)
wstep42x=np.asarray(wstep42x)
wstep52x=np.asarray(wstep52x)
wstep62x=np.asarray(wstep62x)
wstep2x = []
wstep2x=np.concatenate((wstep12x, wstep22x))
wstep2x=np.concatenate((wstep2x, wstep32x))
wstep2x=np.concatenate((wstep2x, wstep42x))
wstep2x=np.concatenate((wstep2x, wstep52x))
wstep2x=np.concatenate((wstep2x, wstep62x))
wstep2x.shape

## STEP 2 y axis
wstep12y=[]
wstep12y=final_window(step12.accely)
wstep12y.shape

wstep22y=[]
wstep22y=final_window(step22.accely)

wstep32y=[]
wstep32y=final_window(step32.accely)

wstep42y=[]
wstep42y=final_window(step42.accely)

wstep52y=[]
wstep52y=final_window(step52.accely)

wstep62y=[]
wstep62y=final_window(step62.accely)

################################################

wstep12y=np.asarray(wstep12y)
wstep22y=np.asarray(wstep22y)
wstep32y=np.asarray(wstep32y)
wstep42y=np.asarray(wstep42y)
wstep52y=np.asarray(wstep52y)
wstep62y=np.asarray(wstep62y)
wstep2y = []
wstep2y=np.concatenate((wstep12y, wstep22y))
wstep2y=np.concatenate((wstep2y, wstep32y))
wstep2y=np.concatenate((wstep2y, wstep42y))
wstep2y=np.concatenate((wstep2y, wstep52y))
wstep2y=np.concatenate((wstep2y, wstep62y))
wstep2y.shape

## STEP 2 z axis
wstep12z=[]
wstep12z=final_window(step12.accelz)
wstep12z.shape

wstep22z=[]
wstep22z=final_window(step22.accelz)

wstep32z=[]
wstep32z=final_window(step32.accelz)

wstep42z=[]
wstep42z=final_window(step42.accelz)

wstep52z=[]
wstep52z=final_window(step52.accelz)

wstep62z=[]
wstep62z=final_window(step62.accelz)

################################################

wstep12z=np.asarray(wstep12z)
wstep22z=np.asarray(wstep22z)
wstep32z=np.asarray(wstep32z)
wstep42z=np.asarray(wstep42z)
wstep52z=np.asarray(wstep52z)
wstep62z=np.asarray(wstep62z)
wstep2z = []
wstep2z=np.concatenate((wstep12z, wstep22z))
wstep2z=np.concatenate((wstep2z, wstep32z))
wstep2z=np.concatenate((wstep2z, wstep42z))
wstep2z=np.concatenate((wstep2z, wstep52z))
wstep2z=np.concatenate((wstep2z, wstep62z))
wstep2z.shape

(765, 128)

In [17]:
## STEP 3 x axis
wstep13x=[]
wstep13x=final_window(step13.accelx)
wstep13x.shape

wstep23x=[]
wstep23x=final_window(step23.accelx)

wstep33x=[]
wstep33x=final_window(step33.accelx)

wstep43x=[]
wstep43x=final_window(step43.accelx)

wstep53x=[]
wstep53x=final_window(step53.accelx)

wstep63x=[]
wstep63x=final_window(step63.accelx)

#############################################

wstep13x=np.asarray(wstep13x)
wstep23x=np.asarray(wstep23x)
wstep33x=np.asarray(wstep33x)
wstep43x=np.asarray(wstep43x)
wstep53x=np.asarray(wstep53x)
wstep63x=np.asarray(wstep63x)
wstep3x = []
wstep3x=np.concatenate((wstep13x, wstep23x))
wstep3x=np.concatenate((wstep3x, wstep33x))
wstep3x=np.concatenate((wstep3x, wstep43x))
wstep3x=np.concatenate((wstep3x, wstep53x))
wstep3x=np.concatenate((wstep3x, wstep63x))
wstep3x.shape

## STEP 3 y axis
wstep13y=[]
wstep13y=final_window(step13.accely)
wstep13y.shape

wstep23y=[]
wstep23y=final_window(step23.accely)

wstep33y=[]
wstep33y=final_window(step33.accely)

wstep43y=[]
wstep43y=final_window(step43.accely)

wstep53y=[]
wstep53y=final_window(step53.accely)

wstep63y=[]
wstep63y=final_window(step63.accely)

################################################

wstep13y=np.asarray(wstep13y)
wstep23y=np.asarray(wstep23y)
wstep33y=np.asarray(wstep33y)
wstep43y=np.asarray(wstep43y)
wstep53y=np.asarray(wstep53y)
wstep63y=np.asarray(wstep63y)
wstep3y = []
wstep3y=np.concatenate((wstep13y, wstep23y))
wstep3y=np.concatenate((wstep3y, wstep33y))
wstep3y=np.concatenate((wstep3y, wstep43y))
wstep3y=np.concatenate((wstep3y, wstep53y))
wstep3y=np.concatenate((wstep3y, wstep63y))
wstep3y.shape

## STEP 3 z axis
wstep13z=[]
wstep13z=final_window(step13.accelz)
wstep13z.shape

wstep23z=[]
wstep23z=final_window(step23.accelz)

wstep33z=[]
wstep33z=final_window(step33.accelz)

wstep43z=[]
wstep43z=final_window(step43.accelz)

wstep53z=[]
wstep53z=final_window(step53.accelz)

wstep63z=[]
wstep63z=final_window(step63.accelz)

################################################

wstep13z=np.asarray(wstep13z)
wstep23z=np.asarray(wstep23z)
wstep33z=np.asarray(wstep33z)
wstep43z=np.asarray(wstep43z)
wstep53z=np.asarray(wstep53z)
wstep63z=np.asarray(wstep63z)
wstep3z = []
wstep3z=np.concatenate((wstep13z, wstep23z))
wstep3z=np.concatenate((wstep3z, wstep33z))
wstep3z=np.concatenate((wstep3z, wstep43z))
wstep3z=np.concatenate((wstep3z, wstep53z))
wstep3z=np.concatenate((wstep3z, wstep63z))
wstep3z.shape

(765, 128)

In [18]:
## STEP 4 x axis
wstep14x=[]
wstep14x=final_window(step14.accelx)
wstep14x.shape

wstep24x=[]
wstep24x=final_window(step24.accelx)

wstep34x=[]
wstep34x=final_window(step34.accelx)

wstep44x=[]
wstep44x=final_window(step44.accelx)

wstep54x=[]
wstep54x=final_window(step54.accelx)

wstep64x=[]
wstep64x=final_window(step64.accelx)

#############################################

wstep14x=np.asarray(wstep14x)
wstep24x=np.asarray(wstep24x)
wstep34x=np.asarray(wstep34x)
wstep44x=np.asarray(wstep44x)
wstep54x=np.asarray(wstep54x)
wstep64x=np.asarray(wstep64x)
wstep4x = []
wstep4x=np.concatenate((wstep14x, wstep24x))
wstep4x=np.concatenate((wstep4x, wstep34x))
wstep4x=np.concatenate((wstep4x, wstep44x))
wstep4x=np.concatenate((wstep4x, wstep54x))
wstep4x=np.concatenate((wstep4x, wstep64x))
wstep4x.shape

## STEP 4 y axis
wstep14y=[]
wstep14y=final_window(step14.accely)
wstep14y.shape

wstep24y=[]
wstep24y=final_window(step24.accely)

wstep34y=[]
wstep34y=final_window(step34.accely)

wstep44y=[]
wstep44y=final_window(step44.accely)

wstep54y=[]
wstep54y=final_window(step54.accely)

wstep64y=[]
wstep64y=final_window(step64.accely)

################################################

wstep14y=np.asarray(wstep14y)
wstep24y=np.asarray(wstep24y)
wstep34y=np.asarray(wstep34y)
wstep44y=np.asarray(wstep44y)
wstep54y=np.asarray(wstep54y)
wstep64y=np.asarray(wstep64y)
wstep4y = []
wstep4y=np.concatenate((wstep14y, wstep24y))
wstep4y=np.concatenate((wstep4y, wstep34y))
wstep4y=np.concatenate((wstep4y, wstep44y))
wstep4y=np.concatenate((wstep4y, wstep54y))
wstep4y=np.concatenate((wstep4y, wstep64y))
wstep4y.shape

## STEP 4 z axis
wstep14z=[]
wstep14z=final_window(step14.accelz)
wstep14z.shape

wstep24z=[]
wstep24z=final_window(step24.accelz)

wstep34z=[]
wstep34z=final_window(step34.accelz)

wstep44z=[]
wstep44z=final_window(step44.accelz)

wstep54z=[]
wstep54z=final_window(step54.accelz)

wstep64z=[]
wstep64z=final_window(step64.accelz)

################################################

wstep14z=np.asarray(wstep14z)
wstep24z=np.asarray(wstep24z)
wstep34z=np.asarray(wstep34z)
wstep44z=np.asarray(wstep44z)
wstep54z=np.asarray(wstep54z)
wstep64z=np.asarray(wstep64z)
wstep4z = []
wstep4z=np.concatenate((wstep14z, wstep24z))
wstep4z=np.concatenate((wstep4z, wstep34z))
wstep4z=np.concatenate((wstep4z, wstep44z))
wstep4z=np.concatenate((wstep4z, wstep54z))
wstep4z=np.concatenate((wstep4z, wstep64z))
wstep4z.shape

(765, 128)

In [19]:
## STEP 5 x axis
wstep15x=[]
wstep15x=final_window(step15.accelx)
wstep15x.shape

wstep25x=[]
wstep25x=final_window(step25.accelx)

wstep35x=[]
wstep35x=final_window(step35.accelx)

wstep45x=[]
wstep45x=final_window(step45.accelx)

wstep55x=[]
wstep55x=final_window(step55.accelx)

wstep65x=[]
wstep65x=final_window(step65.accelx)

#############################################

wstep15x=np.asarray(wstep15x)
wstep25x=np.asarray(wstep25x)
wstep35x=np.asarray(wstep35x)
wstep45x=np.asarray(wstep45x)
wstep55x=np.asarray(wstep55x)
wstep65x=np.asarray(wstep65x)
wstep5x = []
wstep5x=np.concatenate((wstep15x, wstep25x))
wstep5x=np.concatenate((wstep5x, wstep35x))
wstep5x=np.concatenate((wstep5x, wstep45x))
wstep5x=np.concatenate((wstep5x, wstep55x))
wstep5x=np.concatenate((wstep5x, wstep65x))
wstep5x.shape

## STEP 5 y axis
wstep15y=[]
wstep15y=final_window(step15.accely)
wstep15y.shape

wstep25y=[]
wstep25y=final_window(step25.accely)

wstep35y=[]
wstep35y=final_window(step35.accely)

wstep45y=[]
wstep45y=final_window(step45.accely)

wstep55y=[]
wstep55y=final_window(step55.accely)

wstep65y=[]
wstep65y=final_window(step65.accely)

################################################

wstep15y=np.asarray(wstep15y)
wstep25y=np.asarray(wstep25y)
wstep35y=np.asarray(wstep35y)
wstep45y=np.asarray(wstep45y)
wstep55y=np.asarray(wstep55y)
wstep65y=np.asarray(wstep65y)
wstep5y = []
wstep5y=np.concatenate((wstep15y, wstep25y))
wstep5y=np.concatenate((wstep5y, wstep35y))
wstep5y=np.concatenate((wstep5y, wstep45y))
wstep5y=np.concatenate((wstep5y, wstep55y))
wstep5y=np.concatenate((wstep5y, wstep65y))
wstep5y.shape

## STEP 5 z axis
wstep15z=[]
wstep15z=final_window(step15.accelz)
wstep15z.shape

wstep25z=[]
wstep25z=final_window(step25.accelz)

wstep35z=[]
wstep35z=final_window(step35.accelz)

wstep45z=[]
wstep45z=final_window(step45.accelz)

wstep55z=[]
wstep55z=final_window(step55.accelz)

wstep65z=[]
wstep65z=final_window(step65.accelz)

################################################

wstep15z=np.asarray(wstep15z)
wstep25z=np.asarray(wstep25z)
wstep35z=np.asarray(wstep35z)
wstep45z=np.asarray(wstep45z)
wstep55z=np.asarray(wstep55z)
wstep65z=np.asarray(wstep65z)
wstep5z = []
wstep5z=np.concatenate((wstep15z, wstep25z))
wstep5z=np.concatenate((wstep5z, wstep35z))
wstep5z=np.concatenate((wstep5z, wstep45z))
wstep5z=np.concatenate((wstep5z, wstep55z))
wstep5z=np.concatenate((wstep5z, wstep65z))
wstep5z.shape

(765, 128)

In [20]:
## STEP 6 x axis
wstep16x=[]
wstep16x=final_window(step16.accelx)
wstep16x.shape

wstep26x=[]
wstep26x=final_window(step26.accelx)

wstep36x=[]
wstep36x=final_window(step36.accelx)

wstep46x=[]
wstep46x=final_window(step46.accelx)

wstep56x=[]
wstep56x=final_window(step56.accelx)

wstep66x=[]
wstep66x=final_window(step66.accelx)

#############################################

wstep16x=np.asarray(wstep16x)
wstep26x=np.asarray(wstep26x)
wstep36x=np.asarray(wstep36x)
wstep46x=np.asarray(wstep46x)
wstep56x=np.asarray(wstep56x)
wstep66x=np.asarray(wstep66x)
wstep6x = []
wstep6x=np.concatenate((wstep16x, wstep26x))
wstep6x=np.concatenate((wstep6x, wstep36x))
wstep6x=np.concatenate((wstep6x, wstep46x))
wstep6x=np.concatenate((wstep6x, wstep56x))
wstep6x=np.concatenate((wstep6x, wstep66x))
wstep6x.shape

## STEP 6 y axis
wstep16y=[]
wstep16y=final_window(step16.accely)
wstep16y.shape

wstep26y=[]
wstep26y=final_window(step26.accely)

wstep36y=[]
wstep36y=final_window(step36.accely)

wstep46y=[]
wstep46y=final_window(step46.accely)

wstep56y=[]
wstep56y=final_window(step56.accely)

wstep66y=[]
wstep66y=final_window(step66.accely)

################################################

wstep16y=np.asarray(wstep16y)
wstep26y=np.asarray(wstep26y)
wstep36y=np.asarray(wstep36y)
wstep46y=np.asarray(wstep46y)
wstep56y=np.asarray(wstep56y)
wstep66y=np.asarray(wstep66y)
wstep6y = []
wstep6y=np.concatenate((wstep16y, wstep26y))
wstep6y=np.concatenate((wstep6y, wstep36y))
wstep6y=np.concatenate((wstep6y, wstep46y))
wstep6y=np.concatenate((wstep6y, wstep56y))
wstep6y=np.concatenate((wstep6y, wstep66y))
wstep6y.shape

## STEP 6 z axis
wstep16z=[]
wstep16z=final_window(step16.accelz)
wstep16z.shape

wstep26z=[]
wstep26z=final_window(step26.accelz)

wstep36z=[]
wstep36z=final_window(step36.accelz)

wstep46z=[]
wstep46z=final_window(step46.accelz)

wstep56z=[]
wstep56z=final_window(step56.accelz)

wstep66z=[]
wstep66z=final_window(step66.accelz)

################################################

wstep16z=np.asarray(wstep16z)
wstep26z=np.asarray(wstep26z)
wstep36z=np.asarray(wstep36z)
wstep46z=np.asarray(wstep46z)
wstep56z=np.asarray(wstep56z)
wstep66z=np.asarray(wstep66z)
wstep6z = []
wstep6z=np.concatenate((wstep16z, wstep26z))
wstep6z=np.concatenate((wstep6z, wstep36z))
wstep6z=np.concatenate((wstep6z, wstep46z))
wstep6z=np.concatenate((wstep6z, wstep56z))
wstep6z=np.concatenate((wstep6z, wstep66z))
wstep6z.shape

(766, 128)

In [21]:
activities_description = {
    1: 'Step1',
    2: 'Step2',
    3: 'Step3',
    4: 'Step4',
    5: 'Step5',
    6: 'Step6'
}


In [22]:
def randomize(dataset, labels):
    permutation = np.random.permutation(labels.shape[0])
    shuffled_dataset = dataset[permutation, :, :]
    shuffled_labels = labels[permutation]
    return shuffled_dataset, shuffled_labels

In [65]:
def randomize_data(dataset):
    permutation = np.random.permutation(dataset.shape[0])
    shuffled_dataset = dataset[permutation, :, :]
    return shuffled_dataset

In [76]:
step1=np.stack((wstep1x,wstep1y,wstep1z))
step1.shape

(3, 767, 128)

In [77]:
step1_t=step1.transpose(1,2,0)
step1_t.shape

(767, 128, 3)

In [73]:
# step1_t[0]

In [72]:
# step1_t[0]

In [78]:
step2=np.stack((wstep2x,wstep2y,wstep2z))
step2_t=step2.transpose(1,2,0)

step3=np.stack((wstep3x,wstep3y,wstep3z))
step3_t=step2.transpose(1,2,0)

step4=np.stack((wstep4x,wstep4y,wstep4z))
step4_t=step2.transpose(1,2,0)

step5=np.stack((wstep5x,wstep5y,wstep5z))
step5_t=step2.transpose(1,2,0)

step6=np.stack((wstep6x,wstep6y,wstep6z))
step6_t=step2.transpose(1,2,0)

In [79]:
step1_t = randomize_data(step1_t)
step1_t.shape

step2_t = randomize_data(step2_t)
step2_t.shape

step3_t = randomize_data(step3_t)
step3_t.shape

step4_t = randomize_data(step4_t)
step4_t.shape

step5_t = randomize_data(step5_t)
step5_t.shape

step6_t = randomize_data(step6_t)
step6_t.shape

(765, 128, 3)

In [80]:
#537 data points in train
x_train1=step1_t[0:537]
x_test1=step1_t[538:-2]


In [81]:
x_test1.shape

(227, 128, 3)

In [82]:
x_train2=step2_t[:537]
x_test2=step2_t[538:]

x_train3=step3_t[:537]
x_test3=step3_t[538:]

x_train4=step4_t[:537]
x_test4=step4_t[538:]

x_train5=step5_t[:537]
x_test5=step5_t[538:]

x_train6=step6_t[:537]
x_test6=step6_t[538:]

In [51]:
print(type(x_train2))

<class 'numpy.ndarray'>


In [83]:
x_train=[]
x_train=np.concatenate((x_train1, x_train2))
x_train=np.concatenate((x_train, x_train3))
x_train=np.concatenate((x_train, x_train4))
x_train=np.concatenate((x_train, x_train5))
x_train=np.concatenate((x_train, x_train6))
x_train.shape

(3222, 128, 3)

In [84]:
x_test=[]
x_test=np.concatenate((x_test1, x_test2))
x_test=np.concatenate((x_test, x_test3))
x_test=np.concatenate((x_test, x_test4))
x_test=np.concatenate((x_test, x_test5))
x_test=np.concatenate((x_test, x_test6))
x_test.shape

(1362, 128, 3)

In [85]:
y_train1=np.full((537), 1, dtype=int)
y_train2=np.full((537), 2, dtype=int)
y_train3=np.full((537), 3, dtype=int)
y_train4=np.full((537), 4, dtype=int)
y_train5=np.full((537), 5, dtype=int)
y_train6=np.full((537), 6, dtype=int)

y_test1=np.full((227), 1, dtype=int)
y_test2=np.full((227), 2, dtype=int)
y_test3=np.full((227), 3, dtype=int)
y_test4=np.full((227), 4, dtype=int)
y_test5=np.full((227), 5, dtype=int)
y_test6=np.full((227), 6, dtype=int)

In [86]:
y_train=[]
y_train=np.concatenate((y_train1, y_train2))
y_train=np.concatenate((y_train, y_train3))
y_train=np.concatenate((y_train, y_train4))
y_train=np.concatenate((y_train, y_train5))
y_train=np.concatenate((y_train, y_train6))
y_train.shape

(3222,)

In [87]:
y_test=[]
y_test=np.concatenate((y_test1, y_test2))
y_test=np.concatenate((y_test, y_test3))
y_test=np.concatenate((y_test, y_test4))
y_test=np.concatenate((y_test, y_test5))
y_test=np.concatenate((y_test, y_test6))
y_test.shape

(1362,)

In [88]:
signals_train, labels_train = randomize(x_train, y_train)
signals_test, labels_test = randomize(x_test, y_test)

In [62]:
dict_classifiers = {
    "Gradient Boosting Classifier": GradientBoostingClassifier(),
    "Random Forest": RandomForestClassifier(),
    "Logistic Regression": LogisticRegression(),
    "Nearest Neighbors": KNeighborsClassifier(),
    "Decision Tree": DecisionTreeClassifier(),
    "Linear SVM": SVC(),
    "Neural Net": MLPClassifier(alpha = 1),
    "Naive Bayes": GaussianNB(),   
    "AdaBoost": AdaBoostClassifier(),
    "Gaussian Process": GaussianProcessClassifier()
}
   
def batch_classify(X_train, Y_train, X_test, Y_test, no_classifiers = 5, verbose = True):
    """
    This method, takes as input the X, Y matrices of the Train and Test set.
    And fits them on all of the Classifiers specified in the dict_classifier.
    Usually, the SVM, Random Forest and Gradient Boosting Classifier take quiet some time to train. 
    So it is best to train them on a smaller dataset first and 
    decide whether you want to comment them out or not based on the test accuracy score.
    """
    
    dict_models = {}
    for classifier_name, classifier in list(dict_classifiers.items())[:no_classifiers]:
        t_start = time.clock()
        classifier.fit(X_train, Y_train)
        t_end = time.clock()
        
        t_diff = t_end - t_start
        train_score = classifier.score(X_train, Y_train)
        test_score = classifier.score(X_test, Y_test)
        
        dict_models[classifier_name] = {'model': classifier, 'train_score': train_score, 'test_score': test_score, 'train_time': t_diff}
        if verbose:
            print("trained {c} in {f:.2f} s".format(c=classifier_name, f=t_diff))
    return dict_models

def get_train_test(df, y_col, x_cols, ratio):
    """ 
    This method transforms a dataframe into a train and test set, for this you need to specify:
    1. the ratio train : test (usually 0.7)
    2. the column with the Y_values
    """
    mask = np.random.rand(len(df)) < ratio
    df_train = df[mask]
    df_test = df[~mask]
       
    Y_train = df_train[y_col].values
    Y_test = df_test[y_col].values
    X_train = df_train[x_cols].values
    X_test = df_test[x_cols].values
    return df_train, df_test, X_train, Y_train, X_test, Y_test
    
def display_dict_models(dict_models, sort_by='test_score'):
    cls = [key for key in dict_models.keys()]
    test_s = [dict_models[key]['test_score'] for key in cls]
    training_s = [dict_models[key]['train_score'] for key in cls]
    training_t = [dict_models[key]['train_time'] for key in cls]
    
    df_ = pd.DataFrame(data=np.zeros(shape=(len(cls),4)), columns = ['classifier', 'train_score', 'test_score', 'train_time'])
    for ii in range(0,len(cls)):
        df_.loc[ii, 'classifier'] = cls[ii]
        df_.loc[ii, 'train_score'] = training_s[ii]
        df_.loc[ii, 'test_score'] = test_s[ii]
        df_.loc[ii, 'train_time'] = training_t[ii]
    
    display(df_.sort_values(by=sort_by, ascending=False))
    
def calculate_entropy(list_values):
    counter_values = Counter(list_values).most_common()
    probabilities = [elem[1]/len(list_values) for elem in counter_values]
    entropy=scipy.stats.entropy(probabilities)
    return entropy

def calculate_statistics(list_values):
    n5 = np.nanpercentile(list_values, 5)
    n25 = np.nanpercentile(list_values, 25)
    n75 = np.nanpercentile(list_values, 75)
    n95 = np.nanpercentile(list_values, 95)
    median = np.nanpercentile(list_values, 50)
    mean = np.nanmean(list_values)
    std = np.nanstd(list_values)
    var = np.nanvar(list_values)
    rms = np.nanmean(np.sqrt(list_values**2))
    return [n5, n25, n75, n95, median, mean, std, var, rms]

def calculate_crossings(list_values):
    zero_crossing_indices = np.nonzero(np.diff(np.array(list_values) > 0))[0]
    no_zero_crossings = len(zero_crossing_indices)
    mean_crossing_indices = np.nonzero(np.diff(np.array(list_values) > np.nanmean(list_values)))[0]
    no_mean_crossings = len(mean_crossing_indices)
    return [no_zero_crossings, no_mean_crossings]

def get_features(list_values):
    entropy = calculate_entropy(list_values)
    crossings = calculate_crossings(list_values)
    statistics = calculate_statistics(list_values)
    return [entropy] + crossings + statistics

def get_uci_har_features(dataset, labels, waveletname):
    uci_har_features = []
    for signal_no in range(0, len(dataset)):
        features = []
        for signal_comp in range(0,dataset.shape[2]):
            signal = dataset[signal_no, :, signal_comp]
            list_coeff = pywt.wavedec(signal, waveletname)
            for coeff in list_coeff:
                features += get_features(coeff)
        uci_har_features.append(features)
    X = np.array(uci_har_features)
    Y = np.array(labels)
    return X, Y

In [89]:
waveletname = 'db8'
X_train, Y_train = get_uci_har_features(signals_train, labels_train, waveletname)
X_test, Y_test = get_uci_har_features(signals_test, labels_test, waveletname)

In [90]:
models = batch_classify(X_train, Y_train, X_test, Y_test)
display_dict_models(models)

C:\Users\ankit\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
C:\Users\ankit\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


trained Gradient Boosting Classifier in 42.75 s


C:\Users\ankit\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
C:\Users\ankit\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
C:\Users\ankit\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
C:\Users\ankit\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\ankit\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be chan

trained Random Forest in 0.56 s


C:\Users\ankit\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
C:\Users\ankit\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
C:\Users\ankit\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


trained Logistic Regression in 19.58 s
trained Nearest Neighbors in 0.06 s


C:\Users\ankit\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


trained Decision Tree in 1.01 s


C:\Users\ankit\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


,classifier,train_score,test_score,train_time
2,Logistic Regression,0.345127,0.229809,19.580023
3,Nearest Neighbors,0.376785,0.176946,0.059220
0,Gradient Boosting Classifier,0.400062,0.154919,42.751068
1,Random Forest,0.404097,0.152717,0.559598
4,Decision Tree,0.404097,0.142438,1.013840
